In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [2]:
path = '../../../MyDataSets/回归数据集/Bike-Sharing-Dataset/day.csv'

df = pd.read_csv(path)

In [136]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### 1.选择所需要的特征列
### 2.拆分数据集
### 3.数据预处理
### 4.模型训练
### 5.模型评估

In [3]:
data = df.rename(columns={
    'yr':'year',
    'mnth':'month',
    'holiday':'isholiday',
    'workingday':'isworkingday',
    'weathersit':'weather',
    'cnt':'total'
})

myfeaturename = ['season','year','month','isholiday','weekday','isworkingday','weather','temp','atemp','hum','windspeed']

x_train,x_test,y_train,y_test = train_test_split(data[myfeaturename],data.total,random_state=42)

In [243]:
x_train.head()

,season,year,month,isholiday,weekday,isworkingday,weather,temp,atemp,hum,windspeed
688,4,1,11,0,1,1,2,0.380833,0.375621,0.623333,0.235067
649,4,1,10,0,4,1,1,0.435000,0.431167,0.463333,0.181596
637,4,1,9,0,6,0,1,0.542500,0.530296,0.542917,0.227604
525,2,1,6,0,6,0,1,0.710833,0.648367,0.437083,0.144287
367,1,1,1,0,2,1,1,0.150000,0.126275,0.441250,0.365671


In [244]:
x_train.shape

(548, 11)

In [6]:
cat_featurename = ['season','year','month','weekday','weather']

tranfer_result = fix_tranform_oht(x_train,cat_featurename)

In [7]:
x_train_deal = x_train.reset_index() #索引要重排一下
x_train_deal.drop("index",axis=1,inplace=True)
for i in tranfer_result:
    if i in cat_featurename:
        x_train_deal = pd.concat([x_train_deal,tranfer_result[i]],axis=1)

In [8]:
#x = [i for i in x_train_deal.columns if i not in myfeaturename]
x_train_ready =  x_train_deal[[i for i in x_train_deal.columns if i not in cat_featurename]]

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [258]:
tree_estimator = DecisionTreeRegressor()
tree_estimator.fit(x_train_ready,y_train)

score = tree_estimator.score(x_train_ready,y_train)
r2_score = cross_val_score(tree_estimator,x_train_ready,y=y_train,cv=10)
mse_score = cross_val_score(tree_estimator,x_train_ready,y=y_train,cv=10,scoring="neg_mean_squared_error")
print("决策树模型score=",score)
print("决策树模型十折:r2_score=",r2_score)
print("决策树模型十折rmse_score=",np.mean(np.sqrt(-mse_score)))

决策树模型score= 1.0
决策树模型十折:r2_score= [0.8255097  0.73808665 0.70182511 0.79982673 0.58754349 0.70251554
 0.59225168 0.70751163 0.71326858 0.69574538]
决策树模型十折rmse_score= 992.4252483837921


In [252]:
score

array([0.86775828, 0.85534066, 0.83366121, 0.90132518, 0.79705795,
       0.8566601 , 0.78494777, 0.81804918, 0.89162369, 0.88821899])

In [10]:
from sklearn.ensemble import RandomForestRegressor

RF_estimator = RandomForestRegressor()
RF_estimator.fit(x_train_ready,y_train)

score = RF_estimator.score(x_train_ready,y_train)

r2_score = cross_val_score(RF_estimator,x_train_ready,y=y_train,cv=10)
#mse_score = cross_val_score(RF_estimator,x_train_ready,y=y_train,cv=10,scoring="neg_mean_squared_error")
print("随机森林模型score=",score)
print("随机森林模型十折:r2_score=",r2_score)
#print("决策树模型十折rmse_score=",np.mean(np.sqrt(-mse_score)))

随机森林模型score= 0.9811402269627536
随机森林模型十折:r2_score= [0.86609529 0.86546247 0.83243836 0.91171018 0.80417854 0.85296081
 0.78231297 0.82849463 0.89615888 0.88179069]


### 有模型了，开始对测试集进行预测
### 先对测试集数据进行预处理

In [11]:
def fx(df,dic_onehot):
    dic_temp = {}
    for i in dic_onehot:
        if 'scaler_' in i:
            scaler = dic_onehot[i]
            col_name = i.split('_')[1]
            feature_arr = scaler.fit_transform(df[[col_name]])
            if feature_arr.shape[1] > 1:        
                fearure_labels = ['is_'+col_name+'_'+str(cls_label) for cls_label in scaler.classes_]
            else:
                fearure_labels = ['is_'+col_name]    
            df_new = pd.DataFrame(data=feature_arr,columns=fearure_labels)
            dic_temp[str(col_name)]=df_new
    return dic_temp

In [12]:
x_test_onehot = fx(x_test,tranfer_result)

x_test_deal = x_test.reset_index() #索引要重排一下
x_test_deal.drop("index",axis=1,inplace=True)
for i in x_test_onehot:
    if i in cat_featurename:
        x_test_deal = pd.concat([x_test_deal,x_test_onehot[i]],axis=1)

In [13]:
x_test_ready =  x_test_deal[[i for i in x_test_deal.columns if i not in cat_featurename]]

### 终于开始预测啦啦啦！


In [14]:
y_predict = RF_estimator.predict(x_test_ready)

In [15]:
RF_estimator.score(x_test_ready,y_test)

0.8926741547794967

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
rmse = np.sqrt(mean_squared_error(y_test,y_predict))
rmse

643.1381872480605

In [52]:
data=np.array([y_test,y_predict]).T
data
dfshow = pd.DataFrame(data=data,columns=['真实值','预测值'])
dfshow['差值']=dfshow['真实值']-dfshow['预测值']
dfshow.head(20)

,真实值,预测值,差值
0,6606.0,6529.81,76.19
1,1550.0,1575.48,-25.48
2,3747.0,3440.02,306.98
3,6041.0,4954.73,1086.27
4,7538.0,7624.75,-86.75
5,7264.0,7391.46,-127.46
6,1605.0,1461.17,143.83
7,2209.0,2070.44,138.56
8,7499.0,6913.96,585.04
9,5743.0,6415.64,-672.64


In [5]:
def fix_tranform_oht(df,col_names):
    dic_onehot = {}
    for col_name in col_names:
        scaler = LabelBinarizer()
        returnObj = ft_single(df,scaler,col_name)
        dic_onehot[str(returnObj[2])] = returnObj[0]
        dic_onehot["scaler_"+str(returnObj[2])] = returnObj[1]
    return dic_onehot
    


def ft_single(df,scaler,col_name):
    '''
        将某列转化成one-hot编码
        返回
            df_new：新表
            scaler：处理器
            col_name：列名            
    '''
    feature_arr = scaler.fit_transform(df[col_name])
    if feature_arr.shape[1] > 1:        
        fearure_labels = ['is_'+col_name+'_'+str(cls_label) for cls_label in scaler.classes_]
    else:
        fearure_labels = ['is_'+col_name]    
    df_new = pd.DataFrame(data=feature_arr,columns=fearure_labels)
    return df_new,scaler,col_name